In [2]:
import re
import pandas as pd
%matplotlib inline
pd.set_option('display.max_columns', None)


In [46]:
!ls data

ATUS-CPS2014_info.txt atuscps_2014.sps      atusrostec_2014.sas
Activity2014_info.txt atuscps_2014.zip      atusrostec_2014.sps
Resp2014_info.txt     atusintcodebk14.pdf   atusrostec_2014.zip
Rost2014_info.txt     atusresp_2014.dat     atussum_2014.dat
Rostec2014_info.txt   atusresp_2014.do      atussum_2014.do
Sum2014_info.txt      atusresp_2014.sas     atussum_2014.sas
Who2014_info.txt      atusresp_2014.sps     atussum_2014.sps
atusact_2014.dat      atusresp_2014.zip     atussum_2014.zip
atusact_2014.do       atusrost_2014.dat     atususersguide.pdf
atusact_2014.sas      atusrost_2014.do      atuswho_2014.dat
atusact_2014.sps      atusrost_2014.sas     atuswho_2014.do
atusact_2014.zip      atusrost_2014.sps     atuswho_2014.sas
atuscps_2014.dat      atusrost_2014.zip     atuswho_2014.sps
atuscps_2014.do       atusrostec_2014.dat   atuswho_2014.zip
atuscps_2014.sas      atusrostec_2014.do    lexiconnoex2014.pdf


In [3]:
act = pd.read_csv('data/atusact_2014.dat')
cps = pd.read_csv('data/atuscps_2014.dat')
resp = pd.read_csv('data/atusresp_2014.dat')
rost = pd.read_csv('data/atusrost_2014.dat')
rostec = pd.read_csv('data/atusrostec_2014.dat')
sum_ = pd.read_csv('data/atussum_2014.dat') \
                  .rename(columns={'tucaseid':'TUCASEID'})
who = pd.read_csv('data/atuswho_2014.dat')

In [4]:
acts = pd.read_excel('data/lexiconwex2014.xls', 1, header=1, parse_cols=1) \
                    .rename(columns={'6-digit activity code' : 'code',
                                     'Activity' : 'activity'}).dropna()


In [ ]:
{'TUCASEID' : 'case_id', 'TUACTIVITY_N' : 'activity_line_num', 
 'TEWHERE' : 'location', 'TRTCCTOT_LN' : 'sec_child_care_len', 
 'TRTCC_LN',
'TRTCOC_LN', 'TRTEC_LN', 'TRTHH_LN', 'TRTNOHH_LN', 'TRTOHH_LN',
'TRTONHH_LN', 'TRTO_LN', 'TUACTDUR', 'TUACTDUR24', 'TUCC5', 'TUCC5B',
'TUCC7', 'TUCC8', 'TUCUMDUR', 'TUCUMDUR24', 'TUEC24', 'TUSTARTTIM',
'TUSTOPTIME', 'TUTIER1CODE', 'TUTIER2CODE', 'TUTIER3CODE', 'TRCODE',
'TRTIER2', 'TXWHERE'}

In [6]:
act.head()

,TUCASEID,TUACTIVITY_N,TEWHERE,TRTCCTOT_LN,TRTCC_LN,TRTCOC_LN,TRTEC_LN,TRTHH_LN,TRTNOHH_LN,TRTOHH_LN,TRTONHH_LN,TRTO_LN,TUACTDUR,TUACTDUR24,TUCC5,TUCC5B,TUCC7,TUCC8,TUCUMDUR,TUCUMDUR24,TUEC24,TUSTARTTIM,TUSTOPTIME,TUTIER1CODE,TUTIER2CODE,TUTIER3CODE,TRCODE,TRTIER2,TXWHERE
0,20140101140007,1,-1,0,0,0,-1,0,-1,0,-1,0,540,540,1,0,0,97,540,540,-1,04:00:00,13:00:00,1,1,1,10101,101,0
1,20140101140007,2,1,0,0,0,-1,0,-1,0,-1,0,30,30,1,0,0,0,570,570,-1,13:00:00,13:30:00,3,1,1,30101,301,0
2,20140101140007,3,-1,60,60,0,-1,60,-1,60,-1,60,60,60,1,0,0,0,630,630,-1,13:30:00,14:30:00,1,2,1,10201,102,0
3,20140101140007,4,13,10,10,0,-1,10,-1,10,-1,10,10,10,1,0,0,0,640,640,-1,14:30:00,14:40:00,18,11,1,181101,1811,0
4,20140101140007,5,4,30,30,0,-1,30,-1,30,-1,30,30,30,1,0,0,0,670,670,-1,14:40:00,15:10:00,11,1,1,110101,1101,0


## What percent of part-time employees are also students?  Vice Versa?
 - resp 
     - TESCHENR - are you enrolled in high school, college, or university? 
         - (1 - Yes, 2 - No)
     - TELFS - Edited: labor force status
         - (1 - Employed - at work, 2 - Employed - absent, 3 - Unemployed - on layoff, 4 - Unemployed - looking, 5 - Not in labor force)
     - TEHRFTPT - do you usually work more than 35 hours per week at your Respondent File job(s)/family business? 
         - (1 - Yes, 2 - No, 3 - Hours Vary)
     - TESPEMPNOT - Edited: employment status of spouse or unmarried partner
         - (1 - Employed, 2 - Unemployed)
     - TRDPFTPT - Full time or part time employment status of respondent
         - (1 - Full time, 2 - Part time)
     - TRSPFTPT - Full time or part time employment status of spouse or unmarried partner
         - (1 - Full time, 2 - Part time)


## Is there a relationship between weekly wage and care for older/younger relatives?
 - time spent?
 - in home or otherwise for older?
 - time spent working vs. time spent 
 
 - resp
     - TRERNWA - Weekly earnings at main job (2 implied decimals)
         - 0 - 288461 (in cents)
     - TRTEC - Total time spent providing eldercare (in minutes)
         - 0 - 1440 (in minutes)
     - TRTEC_LN - Time spent providing eldercare by activity (in minutes)
         - 0 - 1440 (in minutes)
         - Note: Excludes time spent in activities with codes = 01xxxx or 0805xx
     - TEHRUSLT - Edited: total hours usually worked per week (sum of TEHRUSL1 and Respondent File, TEHRUSL2)
         - 0 - 999 (in hours)
         - -4 (Hours vary)
 - rostec
     - TRELHH - Eldercare recipient is a household member
         - (0 - Recipient is not a household member, 1 - Recipient is a household member)

## Industry worked vs. anything?

## Various activities across prescence of spouse, partner, neither
- resp
    - TRSPPRES - Presence of the respondent's spouse or unmarried partner in the Respondent File, household
        - (1 - Spouse present, 2 - Unmarried partner present, 3 - No spouse or unmarried partner present)
    - 

In [77]:
merged = pd.merge(pd.merge(pd.merge(act, cps), pd.merge(resp, rost))
         , pd.merge(pd.merge(rostec, sum_), who))

## Time spent with work acquaintances outside of work for various industries
- resp
    - TRTCCC - Total nonwork-related time respondent spent with customers, clients, and Respondent File coworkers (in minutes)
         - 
    - TEHRUSLT - Edited: total hours usually worked per week (sum of TEHRUSL1 and Respondent File, TEHRUSL2)
         - 0 - 999 (in minutes)
